In [1]:
import re
import pandas as pd
from collections import OrderedDict

In [2]:
docs_df = pd.read_csv('../../data/law_analyzer_new99.csv')
docs_df

,nama_file,nomor_peraturan,judul,tahun,nomor,konsideran,pasal_1,heading
0,01~pmk.010~2011.json,PMK 01/PMK.010/2011,Perubahan atas Peraturan Menteri Keuangan Nomo...,2011,1,a. bahwa dalam rangka memperoleh perhitungan ...,NaN,[]
1,02~pmk.05~2011.json,PMK 02/PMK.05/2011,Pedoman Akuntansi Dan Pelaporan Aset Berupa Ba...,2011,2,a. bahwa berdasarkan Peraturan Menteri Keuang...,Dalam Peraturan Menteri Keuangan ini yang dima...,"['Pengertian', 'Ruang Lingkup', 'Sistem Akunta..."
2,02~pmk.07~2012.json,PMK 02/PMK.07/2012,Perkiraan Alokasi Dana Bagi Hasil Sumber Daya ...,2012,2,bahwa untuk melaksanakan ketentuan Pasal 27 a...,Alokasi Dana Bagi Hasil Sumber Daya Alam Kehut...,[]
3,03~pmk.011~2012.json,PMK 03/PMK.011/2012,Tata Cara Pengelolaan Dan Pertanggungjawaban F...,2012,3,a. bahwa dalam rangka mendukung usaha pemanfa...,Dalam Peraturan Menteri ini yang dimaksud deng...,"['Tujuan', 'Prinsip', 'Bentuk', 'Ruang Lingkup..."
4,04~pmk.02~2014.json,PMK 04/PMK.02/2014,Perubahan Kedua atas Peraturan Menteri Keuanga...,2014,4,a. bahwa dalam rangka mendukung agar kondisi ...,NaN,[]
...,...,...,...,...,...,...,...,...
1631,uu-9-tahun-1992.json,UU 9 TAHUN 1992,Keimigrasian,1992,9,a. bahwa pengaturan keimigrasian yang meliput...,Dalam Undang-undang ini yang dimaksud dengan:\...,[]
1632,uu-9-tahun-1995.json,UU 9 TAHUN 1995,Usaha Kecil,1995,9,a. bahwa negara Republik Indonesia yang berda...,Dalam Undang-undang ini yang dimaksud dengan:\...,[]
1633,uu-9-tahun-2009.json,UU 9 TAHUN 2009,Badan Hukum Pendidikan,2009,9,a. bahwa untuk mewujudkan fungsi dan tujuan p...,Dalam undang-undang ini yang dimaksud dengan:\...,[]
1634,uu-9-tahun-2015.json,UU 9 TAHUN 2015,Perubahan Kedua atas Undang-Undang Nomor 23 Ta...,2015,9,a. bahwa untuk kesinambungan kepemimpinan di ...,NaN,[]


In [3]:
# eval string in heading column
docs_df['heading'] = docs_df['heading'].apply(eval)

# transform NaN values in pasal_1 column to empty string
docs_df['pasal_1'] = docs_df['pasal_1'].fillna('')

In [4]:
def DetectDefinition(Text):
    Keywords = ["adalah", "merupakan", "yang selanjutnya disebut", "yang selanjutnya disingkat",] #"yang hasil perhitungan"]
    return any(Keyword in Text for Keyword in Keywords)

def ExtractDefinition(Text):
    Parts = Text.split("adalah")
    if len(Parts) == 1:
        Parts = Text.split("merupakan")
    if len(Parts) == 1:
        Parts = Text.split("yang selanjutnya disebut")
    if len(Parts) == 1:
        Parts = Text.split("yang selanjutnya disingkat")
    return Parts[-1].strip()

def DetectDefinitionsInText(Text):
    Sentences = re.split(r'\b\d+\.\s*', Text)
    Sentences = [Sentence.strip()
                for Sentence in Sentences if Sentence.strip()]

    Results = []
    for Sentence in Sentences:
        Sentence = re.sub(r'\s+', ' ', Sentence)

        if DetectDefinition(Sentence):
            Term = None
            if "yang selanjutnya disebut" in Sentence:
                Term = Sentence.split("yang selanjutnya disebut")[0].strip()
            elif "yang selanjutnya disingkat" in Sentence:
                Term = Sentence.split("yang selanjutnya disingkat")[0].strip()
            #elif "yang hasil perhitungan" in Sentence:
            #    Term = Sentence.split("yang hasil perhitungan")[0].strip()
            elif "adalah" in Sentence:
                Term = Sentence.split("adalah")[0].strip()
            elif "merupakan" in Sentence:
                Term = Sentence.split("merupakan")[0].strip()

            Definition = ExtractDefinition(Sentence)

            ShortTerm = None
            if "yang selanjutnya disebut" in Sentence:
                ShortTerm = re.search(
                    r'yang selanjutnya disebut(.*?)adalah', Sentence)
            elif "yang selanjutnya disingkat" in Sentence:
                ShortTerm = re.search(
                    r'yang selanjutnya disingkat(.*?)merupakan', Sentence)

            if ShortTerm:
                ShortTerm = ShortTerm.group(1).strip()

            result = {
                "Text": Sentence,
                "Term": Term,
                "ShortTerm": ShortTerm,
                "Definition": Definition,
                "Type": "DEFINITION"
            }
            Results.append(result)

    return Results

In [5]:
def get_definition_term(pasal_1_content):
    definition_terms = DetectDefinitionsInText(pasal_1_content)
    
    terms = []
    if definition_terms:
        for i in definition_terms:
            if i['Term'] is not None:
                terms.append(i['Term'])
            if i['ShortTerm'] is not None:
                terms.append(i['ShortTerm'])
                
        return terms
    else:
        return None

In [6]:
# apply get_definition_term function to pasal_1 column
docs_df['definition_term'] = docs_df['pasal_1'].apply(get_definition_term)
docs_df

,nama_file,nomor_peraturan,judul,tahun,nomor,konsideran,pasal_1,heading,definition_term
0,01~pmk.010~2011.json,PMK 01/PMK.010/2011,Perubahan atas Peraturan Menteri Keuangan Nomo...,2011,1,a. bahwa dalam rangka memperoleh perhitungan ...,,[],None
1,02~pmk.05~2011.json,PMK 02/PMK.05/2011,Pedoman Akuntansi Dan Pelaporan Aset Berupa Ba...,2011,2,a. bahwa berdasarkan Peraturan Menteri Keuang...,Dalam Peraturan Menteri Keuangan ini yang dima...,"[Pengertian, Ruang Lingkup, Sistem Akuntansi, ...","[Kontraktor Kontrak Kerja Sama,, Barang yang m..."
2,02~pmk.07~2012.json,PMK 02/PMK.07/2012,Perkiraan Alokasi Dana Bagi Hasil Sumber Daya ...,2012,2,bahwa untuk melaksanakan ketentuan Pasal 27 a...,Alokasi Dana Bagi Hasil Sumber Daya Alam Kehut...,[],[Alokasi Dana Bagi Hasil Sumber Daya Alam Kehu...
3,03~pmk.011~2012.json,PMK 03/PMK.011/2012,Tata Cara Pengelolaan Dan Pertanggungjawaban F...,2012,3,a. bahwa dalam rangka mendukung usaha pemanfa...,Dalam Peraturan Menteri ini yang dimaksud deng...,"[Tujuan, Prinsip, Bentuk, Ruang Lingkup, Fasil...",[Badan Usaha Penyedia Jasa Penyiapan Proyek KP...
4,04~pmk.02~2014.json,PMK 04/PMK.02/2014,Perubahan Kedua atas Peraturan Menteri Keuanga...,2014,4,a. bahwa dalam rangka mendukung agar kondisi ...,,[],None
...,...,...,...,...,...,...,...,...,...
1631,uu-9-tahun-1992.json,UU 9 TAHUN 1992,Keimigrasian,1992,9,a. bahwa pengaturan keimigrasian yang meliput...,Dalam Undang-undang ini yang dimaksud dengan:\...,[],"[Keimigrasian, Wilayah Negara Republik Indones..."
1632,uu-9-tahun-1995.json,UU 9 TAHUN 1995,Usaha Kecil,1995,9,a. bahwa negara Republik Indonesia yang berda...,Dalam Undang-undang ini yang dimaksud dengan:\...,[],"[Usaha Kecil, Usaha Menengah dan Usaha Besar, ..."
1633,uu-9-tahun-2009.json,UU 9 TAHUN 2009,Badan Hukum Pendidikan,2009,9,a. bahwa untuk mewujudkan fungsi dan tujuan p...,Dalam undang-undang ini yang dimaksud dengan:\...,[],"[Badan hukum pendidikan, Badan Hukum Pendidika..."
1634,uu-9-tahun-2015.json,UU 9 TAHUN 2015,Perubahan Kedua atas Undang-Undang Nomor 23 Ta...,2015,9,a. bahwa untuk kesinambungan kepemimpinan di ...,,[],None


In [7]:
test_list  = [
    [Umum, Siaga Pencarian dan Pertolongan, Operasi Pencarian dan Pertolongan -  Umum, Operasi Pencarian dan Pertolongan -  Pelaksanaan Operasi Pencarian dan Pertolongan, Operasi Pencarian dan Pertolongan -  Penghentian Pelaksanaan Operasi Pencarian dan Pertolongan, Sistem Informasi, Sistem Komunikasi], 
    
]

In [8]:
# drop pasal_1 column
docs_df = docs_df.drop(columns=['pasal_1'])
docs_df

,nama_file,nomor_peraturan,judul,tahun,nomor,konsideran,heading,definition_term
0,01~pmk.010~2011.json,PMK 01/PMK.010/2011,Perubahan atas Peraturan Menteri Keuangan Nomo...,2011,1,a. bahwa dalam rangka memperoleh perhitungan ...,[],None
1,02~pmk.05~2011.json,PMK 02/PMK.05/2011,Pedoman Akuntansi Dan Pelaporan Aset Berupa Ba...,2011,2,a. bahwa berdasarkan Peraturan Menteri Keuang...,"[Pengertian, Ruang Lingkup, Sistem Akuntansi, ...","[Kontraktor Kontrak Kerja Sama,, Barang yang m..."
2,02~pmk.07~2012.json,PMK 02/PMK.07/2012,Perkiraan Alokasi Dana Bagi Hasil Sumber Daya ...,2012,2,bahwa untuk melaksanakan ketentuan Pasal 27 a...,[],[Alokasi Dana Bagi Hasil Sumber Daya Alam Kehu...
3,03~pmk.011~2012.json,PMK 03/PMK.011/2012,Tata Cara Pengelolaan Dan Pertanggungjawaban F...,2012,3,a. bahwa dalam rangka mendukung usaha pemanfa...,"[Tujuan, Prinsip, Bentuk, Ruang Lingkup, Fasil...",[Badan Usaha Penyedia Jasa Penyiapan Proyek KP...
4,04~pmk.02~2014.json,PMK 04/PMK.02/2014,Perubahan Kedua atas Peraturan Menteri Keuanga...,2014,4,a. bahwa dalam rangka mendukung agar kondisi ...,[],None
...,...,...,...,...,...,...,...,...
1631,uu-9-tahun-1992.json,UU 9 TAHUN 1992,Keimigrasian,1992,9,a. bahwa pengaturan keimigrasian yang meliput...,[],"[Keimigrasian, Wilayah Negara Republik Indones..."
1632,uu-9-tahun-1995.json,UU 9 TAHUN 1995,Usaha Kecil,1995,9,a. bahwa negara Republik Indonesia yang berda...,[],"[Usaha Kecil, Usaha Menengah dan Usaha Besar, ..."
1633,uu-9-tahun-2009.json,UU 9 TAHUN 2009,Badan Hukum Pendidikan,2009,9,a. bahwa untuk mewujudkan fungsi dan tujuan p...,[],"[Badan hukum pendidikan, Badan Hukum Pendidika..."
1634,uu-9-tahun-2015.json,UU 9 TAHUN 2015,Perubahan Kedua atas Undang-Undang Nomor 23 Ta...,2015,9,a. bahwa untuk kesinambungan kepemimpinan di ...,[],None


In [9]:
# save the dataframe to csv
docs_df.to_csv('../../data/law_analyzer_new99_keywords.csv', index=False)